# Camera Calibration

To find the intrinsic and extrinsic parameters of a image formation pipeline. The intrinsic parameters contains information like focal length, principal point and the extrinsic parameters contains information about the rotation and translation of the camera with respect to the world coordinates.

- we collect the world points as xw, yw, zw
- we collect the camera coordinates as xc and yc


In [1]:
import numpy as np

In [2]:
xw=[0, 21.68, 43.36, 65.04, 86.72, 108.4, 130.08]
yw=[21.68, 43.36, 65.04, 86.72, 108.40, 130.08, 0]
#zw = [0]*7
zw=[713.74]*7
xc = [366.9331, 416.9176, 466.7643, 516.8607, 566.9595, 617.385, 678.7745]
yc = [240.6366, 293.9554, 346.9755, 400.1464, 453.5715, 507.2507, 195.8701]

### Sanity check: done here to check number of points

In [15]:
len(xw)

7

## Computing things required for A matrix to solve 
- we compute xwxc, ywxc, zwxc, xwyc, ywyc, and zwyc

In [16]:
#xw,yw,zw,1,0,0,0,0,-xwxc,-ywxc,-zwxc,-xc
#0,0,0,0,xw,yw,zw,1,-xwyc,-ywyc,-zwyc,-yc
xwxc=-np.multiply(np.array(xw),np.array(xc))
ywxc =-np.multiply(np.array(yw),np.array(xc)) 
zwxc=-np.multiply(np.array(zw),np.array(xc))
xwyc=-np.multiply(np.array(xw),np.array(yc))
ywyc=-np.multiply(np.array(yw),np.array(yc))
zwyc=-np.multiply(np.array(zw),np.array(yc))

In [17]:
xwxc

array([    -0.      ,  -9038.773568, -20238.900048, -33616.619928,
       -49166.72784 , -66924.534   , -88294.98696 ])

# Once we have all we need we solve for the matrices

$\begin{bmatrix} u^{i} \\ v^{i} \\ 1 \end{bmatrix}  = \begin{bmatrix} p_{11} & p_{12} & p_{13} & p_{14}  \\ p_{21} & p_{22} & p_{23} & p_{24} \\ p_{31} & p_{32} & p_{33} & p_{34} \end{bmatrix} \begin{bmatrix} x_{w}^{i} \\ y_{w}^{i} \\ z_{w}^{i} \\ 1 \end{bmatrix}$

###### So to solve for this we will have A.p=0

$\begin{bmatrix}xw^{(i)}&yw^{(i)}&zw^{(i)}&1&0& 0 & 0 & 0 & -xc^{(i)}xw^{(i)}&-xc^{(i)}yw^{(i)}&-xc^{(i)}zw^{(i)} &-xc^{(i)}\\ 0&0&0&0&xw^{(i)}&yw^{(i)}&zw^{(i)}&1&-yc^{(i)}xw^{(i)}&-yc^{(i)}yw^{(i)}& -yc^{(i)}zw^{(i)} &-yc^{(i)} \end{bmatrix} \begin{bmatrix} p_{11} \\ p_{12} \\ p_{13} \\ p_{14}  \\ p_{21} \\ p_{22} \\ p_{23} \\ p_{24} \\ p_{31} \\ p_{32} \\ p_{33} \\ p_{34} \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \end{bmatrix} $

###### Now we solve it for almost 7 points which gives us 14 equations for 12 unknowns:

$\begin{bmatrix} xw^{(1)} & yw^{(1)} & zw^{(1)} &1 & 0 & 0 & 0 & 0 & -xc^{(1)}xw^{(1)} & -xc^{(1)}yw^{(1)} & -xc^{(1)}zw^{(1)} & -xc^{(1)} \\ 0 & 0 & 0 & 0 & xw^{(1)} & yw^{(1)} & zw^{(1)}& 1 & -yc^{(1)}xw^{(1)} & -yc^{(1)}yw^{(1)} & -yc^{(1)}zw^{(1)} &-yc^{(1)} \\ ... & ... & ... & ... & ... & ...& ... & ... & ... & ... & ... & ...\\ xw^{(7)} & yw^{(7)} & zw^{(7)} &1 & 0 & 0 & 0 & 0 & -xc^{(7)}xw^{(7)} & -xc^{(7)}yw^{(7)} & -xc^{(7)}zw^{(7)} & -xc^{(7)} \\ 0 & 0 & 0 & 0 & xw^{(7)} & yw^{(7)} & zw^{(7)}& 1 & -yc^{(7)}xw^{(7)} & -yc^{(7)}yw^{(7)} & -yc^{(7)}zw^{(7)} &-yc^{(7)}\end{bmatrix}\begin{bmatrix} p_{11} \\ p_{12} \\ p_{13} \\ p_{14}  \\ p_{21} \\ p_{22} \\ p_{23} \\ p_{24} \\ p_{31} \\ p_{32} \\ p_{33} \\ p_{34} \end{bmatrix} =\begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \end{bmatrix} $

###### we then find the rank of the matrix and take the least eigen values and then the linear combination of their eigenvectors are your projection matrix on reshaping

In [18]:
A=[]
for i in range(0,7):
    mat1 = [xw[i],yw[i],zw[i],1,0,0,0,0,xwxc[i],ywxc[i],zwxc[i],-xc[i]]
    mat2 = [0,0,0,0,xw[i],yw[i],zw[i],1,xwyc[i],ywyc[i],zwyc[i],-yc[i]]
    A.append(mat1)
    A.append(mat2)

In [19]:
A = np.array(A)

In [20]:
A

array([[ 0.00000000e+00,  2.16800000e+01,  7.13740000e+02,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -7.95510961e+03, -2.61894831e+05, -3.66933100e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  2.16800000e+01,
         7.13740000e+02,  1.00000000e+00, -0.00000000e+00,
        -5.21700149e+03, -1.71751967e+05, -2.40636600e+02],
       [ 2.16800000e+01,  4.33600000e+01,  7.13740000e+02,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -9.03877357e+03,
        -1.80775471e+04, -2.97570768e+05, -4.16917600e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  2.16800000e+01,  4.33600000e+01,
         7.13740000e+02,  1.00000000e+00, -6.37295307e+03,
        -1.27459061e+04, -2.09807727e+05, -2.93955400e+02],
       [ 4.33600000e+01,  6.50400000e+01,  7.1374000

In [21]:
A.shape

(14, 12)

In [22]:
inner_product = np.dot(A.T,A)

In [23]:
inner_product

array([[ 4.27720384e+04,  3.29015680e+04,  3.24951547e+05,
         4.55280000e+02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -2.62637143e+07,
        -1.86587092e+07, -1.90768814e+08, -2.67280542e+05],
       [ 3.29015680e+04,  4.27720384e+04,  3.24951547e+05,
         4.55280000e+02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.86587092e+07,
        -2.39265377e+07, -1.73425271e+08, -2.42981017e+05],
       [ 3.24951547e+05,  3.24951547e+05,  3.56597351e+06,
         4.99618000e+03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.90768814e+08,
        -1.73425271e+08, -1.84951493e+09, -2.59130066e+06],
       [ 4.55280000e+02,  4.55280000e+02,  4.99618000e+03,
         7.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -2.67280542e+05,
        -2.42981017e+05, -2.59130066e+06, -3.63059470e+03],
       [ 0.00000000e+00,  0.00000000e+00,  0.0000000

In [24]:
inner_product.shape

(12, 12)

In [25]:
w, v = np.linalg.eig(inner_product)

In [26]:
w

array([ 1.50284271e+12,  4.25117817e+09,  6.30241419e+09,  3.56520226e+06,
        7.36166840e+04,  1.57364399e+04,  9.82206104e+02,  7.79910325e-03,
       -5.23176925e-07,  1.24813856e-08,  1.32458727e-09, -1.11539181e-14])

In [27]:
w/w[0]

array([ 1.00000000e+00,  2.82875789e-03,  4.19366188e-03,  2.37230566e-06,
        4.89849560e-08,  1.04711157e-08,  6.53565471e-10,  5.18956721e-15,
       -3.48124871e-19,  8.30518427e-21,  8.81387830e-22, -7.42187991e-27])

### we take the last 6 eigen values as the rank of the matrix

In [28]:
v 

array([[ 1.28689505e-04, -1.22744911e-03, -3.58834453e-06,
         4.76238488e-02,  3.76191977e-02, -4.50284414e-01,
        -3.15236715e-01,  4.82545225e-01,  6.71782282e-01,
        -1.09768167e-02, -3.45218117e-04, -5.66166506e-05],
       [ 1.17088595e-04,  5.02741245e-04, -8.48073694e-04,
         5.64151780e-02,  1.96640621e-01, -3.61949215e-01,
        -5.58219096e-01,  2.32750788e-01, -6.71797898e-01,
         1.09763194e-02,  3.45237418e-04,  5.66169836e-05],
       [ 1.24246883e-03,  2.04748045e-03,  3.71147599e-03,
         5.55189348e-01,  7.56493500e-01,  1.19382272e-01,
         2.84481005e-01,  1.54446844e-01,  2.03545598e-02,
        -1.72599086e-03, -2.37029565e-06, -1.51988621e-06],
       [ 1.74078632e-06,  2.86866430e-06,  5.20003921e-06,
         7.77859372e-04,  1.05990066e-03,  1.67262964e-04,
         3.98577993e-04,  2.16170788e-04,  3.41981985e-02,
         9.93861950e-01, -5.78986166e-03, -1.42598949e-04],
       [ 8.06549781e-05, -2.76456390e-04, -4.4493818

In [29]:
v_prime = v[:, 6]+v[:,7]+v[:,8]+v[:,9]+v[:,10]+v[:,11]

In [30]:
v_prime

array([ 8.27712141e-01, -9.85888033e-01,  4.57552528e-01,  1.02274244e+00,
       -5.23094429e-01,  8.57294209e-01,  2.61344405e-01,  1.00061364e+00,
       -1.48602862e-03, -1.65964338e-03, -5.82320036e-04,  1.24777983e+00])

### we reshape the values into a 3X4 matrix as desired

In [31]:
P = v_prime.reshape((3,4))

In [32]:
P

array([[ 8.27712141e-01, -9.85888033e-01,  4.57552528e-01,
         1.02274244e+00],
       [-5.23094429e-01,  8.57294209e-01,  2.61344405e-01,
         1.00061364e+00],
       [-1.48602862e-03, -1.65964338e-03, -5.82320036e-04,
         1.24777983e+00]])

# we then separate the 3X3 from it to get the intrinsic matrix and Rotation matrix using QR factorization

In [33]:
P[:,:-1]

array([[ 8.27712141e-01, -9.85888033e-01,  4.57552528e-01],
       [-5.23094429e-01,  8.57294209e-01,  2.61344405e-01],
       [-1.48602862e-03, -1.65964338e-03, -5.82320036e-04]])

In [34]:
P[:,:-1].shape

(3, 3)

## Projection matrix QR factorization

In [35]:
Mext, Mint = np.linalg.qr(P[:,:-1])

In [36]:
Mext.shape

(3, 3)

In [37]:
Mint.shape

(3, 3)

In [38]:
Mint

array([[-0.97915136,  1.29139877, -0.24716796],
       [ 0.        , -0.1980418 , -0.4652899 ],
       [ 0.        ,  0.        ,  0.00829818]])

In [39]:
f_x_pixels = Mint[0][0]/P[2][2]

In [40]:
f_x_pixels

1681.466023632946

In [41]:
p_x_pixels = Mint[0][2]/P[2][2]

In [42]:
p_x_pixels

424.4538161663041

In [43]:
p_y_pixels = Mint[1][2]/P[2][2]

In [44]:
p_y_pixels

799.0278056774581

In [45]:
Mext

array([[-0.84533626, -0.53412038,  0.01104678],
       [ 0.53423245, -0.84521085,  0.01463938],
       [ 0.00151767,  0.01827675,  0.99983181]])

# Finding Rotations along X-axis, Y-axis and Z-axis

##### We will consider for our experiment that checkered board was first rotated along X-axis, Y-axis and Z-axis

##### Rotation Matrix along X:

$R_{\theta_x} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & cos\theta_x & -sin\theta_x \\ 0 & sin\theta_x & cos\theta_x\end{bmatrix}$

##### Rotation Matrix along Y:

$R_{\theta_y} = \begin{bmatrix} cos\theta_y & 0 & sin\theta_y \\ 0 & 1 & 0 \\ -sin\theta_y & 0 & cos\theta_y\end{bmatrix}$

##### Rotation Matrix along Z:

$R_{\theta_z} = \begin{bmatrix} cos\theta_z & -sin\theta_z & 0 \\ sin\theta_z & cos\theta_z & 0 \\ 0 & 0 & 1 \end{bmatrix}$

##### Multiplying $R_{\theta_x}$ and $R_{\theta_y}$ and $R_{\theta_z}$ :

$R_{\theta_{xyz}} = \begin{bmatrix} cos\theta_y cos\theta_z & -cos\theta_y sin\theta_z & sin\theta_y \\ sin\theta_x sin\theta_y cos\theta_z + cos\theta_x sin\theta_z & -sin\theta_x sin\theta_y sin\theta_z + cos\theta_x cos\theta_z & -sin\theta_x cos\theta_y \\ -cos\theta_x sin\theta_y cos\theta_z + sin\theta_x sin\theta_z & cos\theta_x sin\theta_y sin\theta_z + sin\theta_x cos\theta_z & cos\theta_x cos\theta_y \end{bmatrix}$

##### So our final rotation matrix is in this form from this $R_{\theta_{xyz}}$ matrix we need to compare with the values in $M_{ext}$ and we will get our angles of rotation at each axis respectively


## To get $\theta_y$

##### We need take $cos^{-1}(R_{\theta_{xyz}}(1)(3))$  and this will give us $\theta_y$. 

##### where (1)(3) correspond to the indices of the rows and colums of the matrix

In [46]:
theta_y = np.arcsin(Mext[0][2])

## Therefore, $\theta_y = 0.011047008729553763^{\circ} \approx 0^{\circ}$ 

In [47]:
theta_y

0.011047008729553763

## To get $\theta_z$

1. We know $\theta_y$. We can use it to get $\theta_z$ from $R_{\theta_{xyz}}(1)(1)$

2. Divide $cos\theta_y$ from $R_{\theta_{xyz}}(1)(1)$ to get $cos\theta_z$

3. Once we get $cos\theta_z$ we can take $cos^{-1}(\theta_z)$ to get $\theta_z$

In [48]:
cos_theta_z = Mext[0][0]/np.cos(theta_y)

In [49]:
cos_theta_z

-0.845387838801787

In [50]:
theta_z = np.arccos(cos_theta_z)

## Therefore, $\theta_z = 2.5780871591054306^{\circ}$ 

In [51]:
theta_z

2.5780871591054306

## To get $\theta_x$

1. We know $\theta_y$. We can use it to get $\theta_x$ from $R_{\theta_{xyz}}(3)(3)$

2. Divide $cos\theta_y$ from $R_{\theta_{xyz}}(3)(3)$ to get $cos\theta_x$

3. Once we get $cos\theta_x$ we can take $cos^{-1}(\theta_x)$ to get $\theta_x$

In [52]:
cos_theta_x = Mext[2][2]/np.cos(theta_y)

In [53]:
cos_theta_x

0.9998928254429142

In [54]:
theta_x = np.arccos(cos_theta_x)

## Therefore, $\theta_x = 0.014640797214501924^{\circ} \approx 0^{\circ}$ 

In [55]:
theta_x

0.014640797214501924

## To find translations:

$ t =  M^{-1}*\begin{bmatrix} P_{14} \\ P_{24} \\ P_{34} \\ P_{44}\end{bmatrix}$

In [56]:
t = np.matmul(np.linalg.inv(Mint),P[:,-1])

In [57]:
t

array([-511.60822828, -358.33473326,  150.36785282])